<a href="https://colab.research.google.com/github/GuilhermePAmorim/AnalisePreditiva/blob/main/Resolucao_Entrega1_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
auto_mpg = fetch_ucirepo(id=9)

# data (as pandas dataframes)
X = auto_mpg.data.features
y = auto_mpg.data.targets

# metadata
print(auto_mpg.metadata)

# variable information
print(auto_mpg.variables)


{'uci_id': 9, 'name': 'Auto MPG', 'repository_url': 'https://archive.ics.uci.edu/dataset/9/auto+mpg', 'data_url': 'https://archive.ics.uci.edu/static/public/9/data.csv', 'abstract': 'Revised from CMU StatLib library, data concerns city-cycle fuel consumption', 'area': 'Other', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 398, 'num_features': 7, 'feature_types': ['Real', 'Categorical', 'Integer'], 'demographics': [], 'target_col': ['mpg'], 'index_col': ['car_name'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1993, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5859H', 'creators': ['R. Quinlan'], 'intro_paper': None, 'additional_info': {'summary': 'This dataset is a slightly modified version of the dataset provided in the StatLib library.  In line with the use by Ross Quinlan (1993) in predicting the attribute "mpg", 8 of the original instances were removed because they had unknown values for th

In [3]:
X

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin
0,307.0,8,130.0,3504,12.0,70,1
1,350.0,8,165.0,3693,11.5,70,1
2,318.0,8,150.0,3436,11.0,70,1
3,304.0,8,150.0,3433,12.0,70,1
4,302.0,8,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...
393,140.0,4,86.0,2790,15.6,82,1
394,97.0,4,52.0,2130,24.6,82,2
395,135.0,4,84.0,2295,11.6,82,1
396,120.0,4,79.0,2625,18.6,82,1


In [4]:
y

,mpg
0,18.0
1,15.0
2,18.0
3,16.0
4,17.0
...,...
393,27.0
394,44.0
395,32.0
396,28.0


In [5]:
X[X['horsepower'].isna()]

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin
32,98.0,4,NaN,2046,19.0,71,1
126,200.0,6,NaN,2875,17.0,74,1
330,85.0,4,NaN,1835,17.3,80,2
336,140.0,4,NaN,2905,14.3,80,1
354,100.0,4,NaN,2320,15.8,81,2
374,151.0,4,NaN,3035,20.5,82,1


In [6]:
X_semna = X.dropna()

In [7]:
X_semna[X_semna["horsepower"].isna()]

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin


In [8]:
import numpy as np
idx_apagar = X[X["horsepower"].isna()].index

In [9]:
y_semna = y[~y.index.isin(idx_apagar)]

In [10]:
y_semna

,mpg
0,18.0
1,15.0
2,18.0
3,16.0
4,17.0
...,...
393,27.0
394,44.0
395,32.0
396,28.0


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az

In [ ]:
sns.pairplot(X_semna)
plt.show()

corr_matrix = X_semna.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
X_semints = X_semna[["displacement", "cylinders", "horsepower", "weight", "acceleration"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X_semints)

X_scaled_df = pd.DataFrame(X_scaled, columns=X_semints.columns)
X_scaled_df["model_year"] = X_semna["model_year"]
X_scaled_df["origin"] = X_semna["origin"]
print(X_scaled_df.head())

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y_semna, test_size=0.3, random_state=42)

X_train




In [ ]:
X_d = X_train["displacement"].values
X_a = X_train["acceleration"].values
X_w = X_train["weight"].values
X_c = X_train["cylinders"].values
X_h = X_train["horsepower"].values

In [ ]:
with pm.Model() as model_all:
  alpha = pm.Normal("alpha",0,1)
  beta_displacement = pm.Normal("b_d",0,1)
  beta_weight = pm.Normal("b_w",0,1)
  beta_cylinders = pm.Normal("b_c",0,1)
  beta_horsepower = pm.Normal("b_h",0,1)
  beta_acc = pm.Normal("b_a",0,1)

  eps = pm.HalfNormal("eps",1)

  mu = alpha + beta_displacement * X_d + beta_weight * X_w + beta_cylinders * X_c + beta_horsepower * X_h + beta_acc * X_a

  likelihood = pm.Normal("y_obs",mu,eps,observed=y_train.mpg.values)

In [ ]:
with model_all:
  trace_model_all_priori = pm.sample_prior_predictive()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
az.plot_ppc(trace_model_all_priori, group='prior', kind='cumulative',ax=axes[0])
az.plot_ppc(trace_model_all_priori, group='prior',ax=axes[1])

In [ ]:
with model_all:
  trace_model_all = pm.sample()

In [ ]:
az.plot_trace(trace_model_all)

In [ ]:
pm.summary(trace_model_all)

In [ ]:
with model_all:
  trace_model_all_post = pm.sample_posterior_predictive(trace_model_all)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
az.plot_ppc(trace_model_all_post, group='posterior', kind='cumulative', ax=axes[0])
az.plot_ppc(trace_model_all_post, group='posterior', ax=axes[1])

In [ ]:
#Ganhou? Faça isso para gerar as predições
with pm.Model() as model_all:
  X_d_c = pm.MutableData("X_d_c",X_d)
  X_w_c = pm.MutableData("X_w_c",X_w)
  X_c_c = pm.MutableData("X_c_c",X_c)
  X_h_c = pm.MutableData("X_h_c",X_h)
  X_a_c = pm.MutableData("X_a_c",X_a)
  y_obs_c = pm.MutableData("y_obs_c",y_train.mpg.values)

  alpha = pm.Normal("alpha",0,1)
  beta_displacement = pm.Normal("b_d",0,1)
  beta_weight = pm.Normal("b_w",0,1)
  beta_cylinders = pm.Normal("b_c",0,1)
  beta_horsepower = pm.Normal("b_h",0,1)
  beta_acc = pm.Normal("b_a",0,1)

  eps = pm.HalfNormal("eps",1)

  mu = alpha + beta_displacement * X_d_c + beta_weight * X_w_c + beta_cylinders * X_c_c + beta_horsepower * X_h_c + beta_acc * X_a_c

  likelihood = pm.Normal("y_obs",mu,eps,observed=y_obs_c)

In [ ]:
pm.model_to_graphviz(model_all)

In [ ]:
with model_all:
  trace_m = pm.sample()

In [ ]:
with model_all:
    pm.set_data({
        "X_a_c": X_test.acceleration.values,
        "X_w_c": X_test.weight.values,
        "X_h_c": X_test.horsepower.values,
        "X_c_c": X_test.cylinders.values,
        "X_d_c": X_test.displacement.values,
        "y_obs_c": y_test.mpg.values
    })
    trace_post_m = pm.sample_posterior_predictive(trace_m)

In [ ]:
preds = trace_post_m.posterior_predictive["y_obs"].mean(["chain","draw"]).to_numpy()

In [ ]:
preds

In [ ]:
rounded_preds = np.round(preds)

df_predictions = pd.DataFrame({
    'Predicted': preds,
    'Actual': y_test.mpg.values
})

df_predictions['Abs_Difference'] = np.abs(df_predictions['Predicted'] - df_predictions['Actual'])

print(df_predictions.head())
print(df_predictions['Abs_Difference'].sum())

